In [38]:
from gensim.models.word2vec import Word2Vec
from gensim.models.phrases import Phrases, Phraser
import os, json, tqdm, re, itertools, nltk, pprint
from collections import Counter
from nltk.corpus import stopwords


In [3]:
def read_file(file_path):
    all_words_from_files = []
    with open (file_path) as file:
        json_content = json.load(file)
        item_count = 0
        for item in json_content:
            if item_count < 10:
                item_count += 1
                text_content = re.sub("<.*?>", "", item["textContent"])
                text_content = text_content.replace('-\n', '')
                text_content = text_content.replace("\n", '')
                
#                 print (text_content)
#             text_content = [word for word in text_content if len(word) > 0]
            all_words_from_files.append(text_content)
    
    return all_words_from_files
       
        
def read_all_judgments_from_2018(path):
    all_judgments = []
    for filename in tqdm.tqdm(os.listdir(path)):
        judgments = read_file(path + filename)
        all_judgments.append(judgments)
    return all_judgments

# We need a stream of sentences
def flatten_judgments(words):
    chain = list(itertools.chain(*words))
    result = []
    for judgment in chain:
        for word in judgment:
            result.append(word)
    return result
        

In [4]:
words = read_all_judgments_from_2018("../data_filtered_2/")

100%|██████████| 11/11 [00:00<00:00, 23.57it/s]


In [5]:
content = []
for judgment in words:
    for word in judgment:
        content.append(word)
content = " ".join(content)

In [26]:
from nltk.tokenize import TweetTokenizer, sent_tokenize
nltk.download('punkt')

tokenizer_words = TweetTokenizer(preserve_case=False)
tokens_sentences = [tokenizer_words.tokenize(t) for t in nltk.sent_tokenize(content)]


[nltk_data] Downloading package punkt to /home/grzegorz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
print(tokens_sentences[:10])

[['uzasadnienie', 'powód', '(', '...', ')', 'sp', '.'], ['z', 'o', '.', 'o', '.'], ['sp', '.'], ['k', '.', 'z', 'siedzibą', 'w', 'p', '.', 'wniósł', 'o', 'zwolnienie', 'spod', 'egzekucji', 'administracyjnej', 'prawa', 'majątkowego', 'przysługującego', 'powodowi', 'w', 'postaci', 'wierzytelności', 'z', 'tytułu', 'nienależnie', 'pobranego', 'podatku', 'akcyzowego', 'za', 'miesiące', 'luty', ',', 'marzec', ',', 'kwiecień', ',', 'maj', ',', 'wrzesień', 'i', 'październik', '2011', 'r', '.', 'określonego', 'w', 'decyzji', 'naczelnika', 'urzędy', 'celnego', 'w', 'z', '.', 'z', 'dnia', '6', 'lutego', '2013', 'r', '.', 'nr', '(', '...', ')', '-', '(', '...', ')', '9110-414', '/', '12/13', '/', 'dn', 'oraz', 'dyrektora', 'izby', 'celnej', 'w', 'r', '.', 'z', 'dnia', '8', 'kwietnia', '2014', 'r', '.', 'nr', '(', '...', ')', '-', '(', '...', ')', '-24/13', '/', '14', ',', 'która', 'został', 'stwierdzona', 'prawomocnym', 'wyrokiem', 'wojewódzkiego', 'sądu', 'administracyjnego', 'w', 'g', '.', 'z', 

In [52]:
bigrams = Phrases(tokens_sentences, threshold=10.0)

In [59]:
bigram_counter = Counter()
for key in bigrams.vocab.keys():
    key_str = str(key)
    splited = key_str.split("_")
    if len(splited)>1:
        bigram_counter[key] += bigrams.vocab[key]

for key, counts in bigram_counter.most_common(100):
    print (str(key) + " " + str(counts))

b'k_.' 39166
b',_\xc5\xbce' 20844
b'art_.' 20079
b'(_...' 15453
b'..._)' 15357
b'r_.' 15096
b'._,' 13399
b'm_.' 12940
b'p_.' 11646
b'._(' 11637
b'z_dnia' 11213
b')_.' 10722
b')_,' 9391
b',_a' 8534
b'c_.' 8331
b'._p' 8292
b'._k' 8239
b'j_.' 8190
b'\xc2\xa7_1' 8064
b'1_)' 7987
b',_i\xc5\xbc' 7531
b',_w' 7519
b'(_1' 7301
b'._c' 7169
b'a_.' 6948
b'._w' 6893
b's_.' 6693
b'w_dniu' 6282
b'd_.' 6208
b'._n' 5914
b'2017_roku' 5829
b'ust_.' 5720
b'na_podstawie' 5648
b'n_.' 5578
b'w_.' 5311
b'z_art' 4887
b'roku_,' 4842
b',_k' 4737
b'w_sprawie' 4639
b'do_pracy' 4156
b'2015_roku' 4093
b'w_tym' 4033
b')_w' 3975
b'._m' 3710
b',_z' 3537
b'podstawie_art' 3518
b',_-' 3513
b',_nie' 3260
b'g_.' 3212
b'1_k' 3175
b'na_k' 3157
b'._d' 3016
b',_co' 2998
b'zw_.' 2970
b'w_okresie' 2928
b'\xe2\x80\x93_k' 2907
b'w_zw' 2882
b'._s' 2861
b'ubezpiecze\xc5\x84_spo\xc5\x82ecznych' 2854
b',_na' 2804
b'od_dnia' 2674
b'2016_roku' 2673
b'z\xc5\x82_,' 2617
b'dzia\xc5\x82alno\xc5\x9bci_gospodarczej' 2595
b'1_kk' 2532
b'(_k' 25

In [22]:
trigrams = Phrases(phrases[tokens_sentences], threshold=1.0)


/home/grzegorz/anaconda2/envs/py3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [64]:
trigram_counter = Counter()
for key in trigrams.vocab.keys():
    key_str = str(key)
    splited = key_str.split("_")
    if len(splited)>2:
        trigram_counter[key] += trigrams.vocab[key]

for key, counts in trigram_counter.most_common(100):
    print (str(unicode(key, "utf-8")) + " " + str(counts))

NameError: name 'unicode' is not defined

In [61]:
sentences_transformed = [trigrams[sentence] for sentence in tokens_sentences]

/home/grzegorz/anaconda2/envs/py3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [63]:
print(sentences_transformed[:10])

[['uzasadnienie', 'powód', '(_...', ')_sp', '.'], ['z', 'o', '.', 'o', '.'], ['sp', '.'], ['k', '.', 'z_siedzibą', 'w', 'p', '.', 'wniósł_o', 'zwolnienie', 'spod_egzekucji', 'administracyjnej', 'prawa', 'majątkowego', 'przysługującego', 'powodowi', 'w_postaci', 'wierzytelności', 'z_tytułu', 'nienależnie_pobranego', 'podatku', 'akcyzowego', 'za_miesiące', 'luty', ',', 'marzec', ',', 'kwiecień', ',', 'maj', ',', 'wrzesień_i', 'październik_2011', 'r', '.', 'określonego_w', 'decyzji', 'naczelnika', 'urzędy', 'celnego', 'w', 'z', '.', 'z_dnia', '6', 'lutego_2013', 'r', '.', 'nr_(', '..._)', '-', '(_...', ')', '9110-414', '/', '12/13', '/', 'dn', 'oraz', 'dyrektora', 'izby', 'celnej', 'w', 'r', '.', 'z_dnia', '8', 'kwietnia_2014', 'r', '.', 'nr_(', '..._)', '-', '(_...', ')', '-24/13', '/', '14', ',', 'która', 'został', 'stwierdzona', 'prawomocnym_wyrokiem', 'wojewódzkiego', 'sądu', 'administracyjnego', 'w', 'g', '.', 'z_dnia', '6', 'lipca_2016', 'r', '.', 'sygn', '.'], ['akt', 'i', 'sa', '/